In [ ]:
import pandas as pd
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# récupérer les fichier du dossier contenant les fihcier de clusters
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_cluster= glob.glob('/content/drive/MyDrive/Colab Notebooks/km/*clean*')

In [ ]:
# vérifier la liste
print(list_cluster)

In [ ]:
# fonction pour lire une liste de fichier 
# argument : une liste de fichiers
# sortie: dictionnaire
def read_cluster(files):
  liste=[]
  for i,cluster in enumerate(files):
    with open(cluster, "r") as file:
      liste.append(file.readlines())
  return liste

In [ ]:
dic= read_cluster(list_cluster)

In [ ]:
print((dic[3]))

In [ ]:
# Mettre chaque dictionnaire dans un dataframe
for i in range(len(list_cluster)):
  vars()["data_"+str(i)]=pd.DataFrame(dic[i], columns = ['tweet'])

In [ ]:
print(data_3.head(10))

In [ ]:
!pip install gensim==4.0

In [ ]:
from gensim.models import KeyedVectors

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
# récupérer les embeddings entrainés sur nos données
w2v = KeyedVectors.load('/content/drive/MyDrive/Colab Notebooks/w2vec_model_d300_1M')

In [ ]:
import numpy as np

In [ ]:
#encode un message
def encode(msg):
    return np.mean([w2v[word] for word in msg if word in w2v] or [np.zeros(300)], axis = 0)

In [ ]:
w2v.similar_by_word("sale")

[('sal', 0.6782066822052002),
 ('pvte', 0.629138708114624),
 ('mèrde', 0.6162537336349487),
 ('pute', 0.6096962094306946),
 ('pte', 0.6044666767120361),
 ('clochard', 0.6014346480369568),
 ('slp', 0.6004031896591187),
 ('crasseux', 0.5957321524620056),
 ('puent', 0.5908592939376831),
 ('daronn', 0.5900101661682129)]

In [79]:
# Encoder un dictionnaire 
def encoded(dic):
  w2v=[]
  for i in range(len(dic)):
    w2v.append(np.array([encode(msg.split()) for msg in dic[i]]))
  return w2v

In [80]:
%%time
#encodage de l'échantillon pour créer le jeu de données
dataset_encoded = encoded(dic)

CPU times: user 25.7 s, sys: 326 ms, total: 26 s
Wall time: 26 s


In [ ]:
print(len(dataset_encoded[0]))

In [82]:
from sklearn.ensemble import IsolationForest

# definir le modèle ainsi que ses paramètres
clf=IsolationForest(n_estimators=100, max_samples='auto', \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)


In [83]:
# Entrainer le modèle
%%time
for i in range(len(dataset_encoded)):
  clf.fit(dataset_encoded[i])

CPU times: user 2min 57s, sys: 3.68 s, total: 3min 1s
Wall time: 1min 35s


In [84]:
#mettre les prédictions dans un dictionnaire
%%time
pred=[]
for i in range(len(dataset_encoded)):
  pred.append(clf.predict(dataset_encoded[i]))

CPU times: user 1min 3s, sys: 526 ms, total: 1min 4s
Wall time: 1min 5s


In [ ]:
print(pred[1])

In [86]:
# ajouter une colonne pred à chaque dataframe de cluster
for i in range(len(dataset_encoded)):
  vars()["data_"+str(i)]['pred']= pred[i]

In [ ]:
print(data_1.head(20))

In [88]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
outliers=[]
for i in range(len(dataset_encoded)):
  outliers.append((vars()["data_"+str(i)].loc[vars()["data_"+str(i)]['pred']==-1]))
  outlier_index=list(outliers[i].index)
  print("cluster"+str(i)+":"+"\n")
  print(vars()["data_"+str(i)]['pred'].value_counts())
  print("\n")

cluster0:

-1    3689
 1     644
Name: pred, dtype: int64


cluster1:

-1    3927
 1     256
Name: pred, dtype: int64


cluster2:

 1    83714
-1     6268
Name: pred, dtype: int64


cluster3:

 1    31858
-1        3
Name: pred, dtype: int64


cluster4:

 1    87389
-1     6111
Name: pred, dtype: int64


cluster5:

 1    7865
-1    6031
Name: pred, dtype: int64


cluster6:

 1    73795
-1     7330
Name: pred, dtype: int64


cluster7:

 1    99312
-1     7959
Name: pred, dtype: int64


cluster8:

 1    38814
-1     8686
Name: pred, dtype: int64


cluster9:

 1    174166
-1     10315
Name: pred, dtype: int64




In [97]:
outliers[3]

,tweet,pred
18792,meinst main\n,-1
20134,métanse al gringasflop\n,-1
21307,désespère mansoor al salmy\n,-1


In [93]:
for i in range(len(outliers)):
  with open(f"drive/MyDrive/outliers/outliers{i}", 'w') as f:
    #dfAsString = outliers[i]['tweet'].to_string(justify='justify')
    dfAsString = str(outliers[i]['tweet'])
    f.write(dfAsString)